# Analysis

### Setup

In [1]:
from env import Hangman, HangmanCheat
from network import NNAgent, Network

In [2]:
policy_network = Network()
policy_network.load_weights('policy.h5') #input p
player = NNAgent(policy_network)
player.eval() #setting to evaluation state so the agent will not memorize play history.
player.reset_guessed()

In [3]:
# test setup
env = Hangman('words/top_1000.txt' , max_lives = 8, verbose = True)

done = False
state = env.reset()

while not done :
    guess = player.select_action(state)
    print('Guessing', guess)
    state, _ , done , _ = env.step(guess)
    
player.reset_guessed()

Guessing a
>> ----- | 7
Guessing e
>> ----- | 6
Guessing o
>> -o--- | 6
Guessing i
>> -o--- | 5
Guessing u
>> -ou-- | 5
Guessing s
>> -ou-- | 4
Guessing n
>> -ou-- | 3
Guessing t
>> -ou-t | 3
Guessing c
>> cou-t | 3
Guessing r
>> court | 3 ! You Win
Word is 'court'


In [14]:
# test cheat setup
env = HangmanCheat('words/top_1000.txt' , max_lives = 8, verbose = True, reject_rate=0.2)

done = False
state = env.reset()

while not done :
    guess = player.select_action(state)
    print('Guessing', guess)
    state, _ , done , _ = env.step(guess)
    
player.reset_guessed()

Guessing e
>> ------- | 8 ! Rejected guess
Guessing a
>> ------- | 7
Guessing o
>> ------- | 6
Guessing i
>> ------- | 5
Guessing u
>> ------- | 5 ! Rejected guess
Guessing y
>> ------- | 4
Guessing s
>> ------- | 4 ! Rejected guess
Guessing h
>> ------- | 3
Guessing r
>> ------- | 2
Guessing l
>> ------- | 1
Guessing n
>> -----n- | 1
Guessing g
>> -----n- | 0 ! You Lose
Word is 'student'


## *Honest* case

Analyze the given policy in the case, where the environment (so the second player) is honest.

Derive how many words are guessed correctly using the top 1000 frequent words for different maximum number of guesses.

In [15]:
from tqdm import tqdm

# results dictionary for max_lives in range 1-10
results = {}

for max_lives in range(1, 11):
    print(f"Testing with max_lives = {max_lives}")
    print("=====================================")

    # (re-)initialize the environment
    env = Hangman('words/top_1000.txt' , max_lives = max_lives, verbose = False)
    player.reset_guessed()

    # count the number of correct guesses
    correct = 0

    # loop through all the words
    for word in tqdm(env.words):
        # reset the environment
        state = env.reset(word)
        done = False
        # play the game
        while not done:
            guess = player.select_action(state)
            state, _ , done , _ = env.step(guess)
        # check if the word is guessed correctly
        if env.is_game_won():
            correct += 1
        player.reset_guessed()
    results[max_lives] = correct / len(env.words)

print(results)

Testing with max_lives = 1


100%|██████████| 1000/1000 [01:30<00:00, 11.11it/s]


Testing with max_lives = 2


100%|██████████| 1000/1000 [02:52<00:00,  5.79it/s]


Testing with max_lives = 3


...

Testing with max_lives = 9


100%|██████████| 1000/1000 [08:05<00:00,  2.06it/s]


Testing with max_lives = 10


100%|██████████| 1000/1000 [08:21<00:00,  2.00it/s]

{1: 0.013, 2: 0.051, 3: 0.106, 4: 0.21, 5: 0.317, 6: 0.434, 7: 0.552, 8: 0.646, 9: 0.721, 10: 0.781}


| Dataset   | max_lives | 0.0   | 0.1   | 0.2   | 0.3   | 0.4   | 0.5   | 0.6   | 0.7   | 0.8   | 0.9   | 1.0   |
|---        |----------:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
| Top 1000  | 1         | 0.013 | 0.005 | 0.005 | 0.002 | 0.001 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
|           | 2         | 0.051 | 0.030 | 0.020 | 0.012 | 0.011 | 0.004 | 0.001 | 0.000 | 0.000 | 0.000 | 0.000 |
|           | 3         | 0.106 | 0.065 | 0.033 | 0.018 | 0.005 | 0.001 | 0.000 | 0.003 | 0.002 | 0.001 | 0.000 |
|           | 4         | 0.210 | 0.127 | 0.070 | 0.040 | 0.019 | 0.013 | 0.006 | 0.003 | 0.000 | 0.000 | 0.000 |
|           | 5         | 0.317 | 0.178 | 0.107 | 0.061 | 0.033 | 0.014 | 0.010 | 0.004 | 0.000 | 0.000 | 0.000 |
|           | 6         | 0.434 | 0.251 | 0.155 | 0.069 | 0.046 | 0.024 | 0.009 | 0.006 | 0.003 | 0.001 | 0.000 |
|           | 7         | 0.552 |       |       |       |       |       |       |       |       |       | 0.000 |
|           | 8         | 0.646 | 0.375 | 0.193 | 0.128 | 0.066 | 0.038 | 0.014 | 0.006 | 0.002 | 0.002 | 0.000 |
|           | 9         | 0.721 |       |       |       |       |       |       |       |       |       | 0.000 |
|           | 10        | 0.781 |       |       |       |       |       |       |       |       |       | 0.000 |

## *Cheat* case

Analyze the given policy in the case, where the environment (so the second player) is cheating.

Keep the number of maximum guesses fixed at first. Derive how many words are guessed correctly using the top 1000 frequent words and vary the cheating factor.

In [ ]:
from tqdm import tqdm

# results dictionary for reject_rate in range 0-1 and max_lives in range 1-10
results = {}

for max_lives in [7,9,10]:#range(1, 11):
    for reject_rate in range(1, 10):
        reject_rate /= 10
        print(f"Testing with max_lives = {max_lives} reject_rate = {reject_rate}")
        print("=====================================")

        # (re-)initialize the environment
        env = HangmanCheat('words/top_1000.txt' , max_lives = max_lives, verbose = False, reject_rate=reject_rate)
        player.reset_guessed()

        # count the number of correct guesses
        correct = 0

        # loop through all the words
        for word in tqdm(env.words):
            # reset the environment
            state = env.reset(word)
            done = False
            # play the game
            while not done:
                guess = player.select_action(state)
                state, _ , done , _ = env.step(guess)
            # check if the word is guessed correctly
            if env.is_game_won():
                correct += 1
            player.reset_guessed()
        results[(max_lives, reject_rate)] = correct / len(env.words)

    print(results)

Testing with max_lives = 7 reject_rate = 0.1


100%|██████████| 1000/1000 [08:09<00:00,  2.04it/s]


Testing with max_lives = 7 reject_rate = 0.2


100%|██████████| 1000/1000 [08:18<00:00,  2.00it/s]


Testing with max_lives = 7 reject_rate = 0.3


...

100%|██████████| 1000/1000 [08:14<00:00,  2.02it/s]


Testing with max_lives = 7 reject_rate = 0.8


100%|██████████| 1000/1000 [08:11<00:00,  2.03it/s]


Testing with max_lives = 7 reject_rate = 0.9


 11%|█▏        | 114/1000 [00:49<06:58,  2.12it/s]